-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Activity by Traffic Lab
Process streaming data to display total active users by traffic source.

##### Objectives
1. Read data stream
2. Get active users by traffic source
3. Execute query with display() and plot results
4. Execute the same streaming query with DataStreamWriter
5. View results being updated in the query table
6. List and stop all active streams

##### Classes
- <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.streaming.DataStreamReader.html#pyspark.sql.streaming.DataStreamReader" target="_blank">DataStreamReader</a>
- <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.streaming.DataStreamWriter.html#pyspark.sql.streaming.DataStreamWriter" target="_blank">DataStreamWriter</a>
- <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.streaming.StreamingQuery.html#pyspark.sql.streaming.StreamingQuery" target="_blank">StreamingQuery</a>

### Setup
Run the cells below to generate data and create the **`schema`** string needed for this lab.

In [0]:
%run ../Includes/Classroom-Setup

Deleted the working directory dbfs:/user/si@elisaandgeeks.com/dbacademy/aspwd/asp_5_1cl_activity_by_traffic_lab


Your working directory is
dbfs:/user/si@elisaandgeeks.com/dbacademy/aspwd

The source for this dataset is
wasbs://courseware@dbacademy.blob.core.windows.net/apache-spark-programming-with-databricks/v02/

Skipping install of existing dataset to
dbfs:/user/si@elisaandgeeks.com/dbacademy/aspwd/datasets


Out[5]: DataFrame[key: string, value: string]

### 1. Read data stream
- Set to process 1 file per trigger
- Read from Delta with filepath stored in **`events_path`**

Assign the resulting DataFrame to **`df`**.

In [0]:
# TODO
df = (spark
      .readStream
      .option("maxFilesPerTrigger", 1)
      .load(events_path)
     )

df.isStreaming

Out[8]: True

**1.1: CHECK YOUR WORK**

In [0]:
assert df.isStreaming
assert df.columns == ["device", "ecommerce", "event_name", "event_previous_timestamp", "event_timestamp", "geo", "items", "traffic_source", "user_first_touch_timestamp", "user_id"]
print("All test pass")

All test pass


### 2. Get active users by traffic source
- Set default shuffle partitions to number of cores on your cluster (not required, but runs faster)
- Group by **`traffic_source`**
  - Aggregate the approximate count of distinct users and alias with "active_users"
- Sort by **`traffic_source`**

In [0]:
# TODO
from pyspark.sql.functions import approx_count_distinct, hour, window

spark.conf.set("spark.sql.shuffle.partitions", spark.sparkContext.defaultParallelism)

traffic_df = df.groupBy("traffic_source").agg(approx_count_distinct("user_id").alias("active_users")).sort("traffic_source")
  

**2.1: CHECK YOUR WORK**

In [0]:
assert str(traffic_df.schema) == "StructType(List(StructField(traffic_source,StringType,true),StructField(active_users,LongType,false)))"
print("All test pass")

All test pass


### 3. Execute query with display() and plot results
- Execute results for **`traffic_df`** using display()
- Plot the streaming query results as a bar graph

In [0]:
# TODO
display(traffic_df)

traffic_source,active_users
direct,438886
email,281525
facebook,956769
google,1781961
instagram,530050
youtube,253321


**3.1: CHECK YOUR WORK**
- You bar chart should plot **`traffic_source`** on the x-axis and **`active_users`** on the y-axis
- The top three traffic sources in descending order should be **`google`**, **`facebook`**, and **`instagram`**.

### 4. Execute the same streaming query with DataStreamWriter
- Name the query "active_users_by_traffic"
- Set to "memory" format and "complete" output mode
- Set a trigger interval of 1 second

In [0]:
# TODO
traffic_query = (traffic_df
                 .writeStream
                 .queryName("active_users_by_traffic")
                 .format("memory")
                 .outputMode("complete")
                 .trigger(processingTime="1 second")
                 .start()
)

**4.1: CHECK YOUR WORK**

In [0]:
until_stream_is_ready("active_users_by_traffic")
assert traffic_query.isActive
assert "active_users_by_traffic" in traffic_query.name
assert traffic_query.lastProgress["sink"]["description"] == "MemorySink"
print("All test pass")

The stream active_users_by_traffic is active and ready.
All test pass


### 5. View results being updated in the query table
Run a query in a SQL cell to display the results from the **`active_users_by_traffic`** table

In [0]:
%sql
SELECT traffic_source, active_users FROM active_users_by_traffic

traffic_source,active_users
direct,438886
email,281525
facebook,956769
google,1781961
instagram,530050
youtube,253321


**5.1: CHECK YOUR WORK**
Your query should eventually result in the following values.

|traffic_source|active_users|
|---|---|
|direct|438886|
|email|281525|
|facebook|956769|
|google|1781961|
|instagram|530050|
|youtube|253321|

### 6. List and stop all active streams
- Use SparkSession to get list of all active streams
- Iterate over the list and stop each query

In [0]:
# TODO
for s in spark.streams.active:
    print(s.name)
    s.stop()

display_query_3
active_users_by_traffic


**6.1: CHECK YOUR WORK**

In [0]:
assert not traffic_query.isActive
print("All test pass")

All test pass


### Classroom Cleanup
Run the cell below to clean up resources.

In [0]:
classroom_cleanup()

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>